## Mask to calculate HSS in tensorflow

$$ f(pixel) =  \begin{cases}
    0,              & \text{if } x < \tau \\
    1,              & \text{otherwise}
\end{cases}, \text{where } \tau = 30  $$

Adapt to paper's HSS defintion, refer to section 5.2 of http://papers.nips.cc/paper/7145-deep-learning-for-precipitation-nowcasting-a-benchmark-and-a-new-model

$$ \text{hss_tau30} = \dfrac {TT * FF - FT * TF} {(TT + FT)*(FT + FF) + (TT + TF)*(TF + FF)} $$

In [1]:
import numpy as np
import tensorflow as tf
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

# global setting
tau = 0.5
N = 501
M = 1

Instructions for updating:
non-resource variables are not supported in the long term


In [2]:
# generate input sample
y_pred = np.random.randn(M, N, N)
y_true = np.random.randn(M, N, N)
assert y_pred.shape == (M, N, N)
assert y_true.shape == (M, N, N)
y_pred_bool_mask = (y_pred > tau)
y_true_bool_mask = (y_true > tau)

TT = y_pred_bool_mask & y_true_bool_mask
TF = y_pred_bool_mask & (~y_true_bool_mask)
FT = (~y_pred_bool_mask) & y_true_bool_mask
FF = (~y_pred_bool_mask) & (~y_true_bool_mask)
TT_v = np.count_nonzero(TT)
TF_v = np.count_nonzero(TF)
FT_v = np.count_nonzero(FT)
FF_v = np.count_nonzero(FF)
print("TT=%d, TF=%d, FT=%s, FF=%d" % (TT_v, TF_v, FT_v, FF_v))
assert TT_v + TF_v + FT_v + FF_v == N * N
hss_score_tau30 = float(TT_v * FF_v - FT_v * TF_v) / float((TT_v+FT_v)*(FT_v+FF_v)+(TT_v+TF_v)*(TF_v+FF_v))
print("hss_score_tau30=%.10f" % (hss_score_tau30))

TT=23957, TF=53462, FT=53664, FF=119918
hss_score_tau30=0.0001446566


In [3]:
# tensorflow for simulation of hss_tau30
tf_y_pred = tf.placeholder(dtype=tf.float32, shape=(None, N, N))
tf_y_true = tf.placeholder(dtype=tf.float32, shape=(None, N, N))

In [4]:
tau = 0.5

def hss_tau30(y_true: tf.Tensor, y_pred: tf.Tensor):
    """
    Using global variable tau to get value of TT, TF, FT, FF.
    see http://www.cawcr.gov.au/projects/verification/?spm=5176.11409106.555.15.50b01e8bEruSg4 for Heidke skill score (Cohen's k) - HSS.
    """
    y_pred_bool_mask = (y_pred >= tau)
    y_true_bool_mask = (y_true >= tau)
    TT = y_pred_bool_mask & y_true_bool_mask
    TF = y_pred_bool_mask & (~y_true_bool_mask)
    FT = (~y_pred_bool_mask) & y_true_bool_mask
    FF = (~y_pred_bool_mask) & (~y_true_bool_mask)
    TT_v = tf.count_nonzero(TT)
    TF_v = tf.count_nonzero(TF)
    FT_v = tf.count_nonzero(FT)
    FF_v = tf.count_nonzero(FF)
    return tf.cast(TT_v * FF_v - FT_v * TF_v, dtype=tf.float32) / tf.cast((TT_v+FT_v)*(FT_v+FF_v)+(TT_v+TF_v)*(TF_v+FF_v), dtype=tf.float32)

In [5]:
tf_hss_tau30 = hss_tau30(tf_y_true, tf_y_pred)

In [6]:
with tf.Session() as sess:
    tf_hss_score_tau30_v = sess.run(tf_hss_tau30, feed_dict={tf_y_pred:y_pred, tf_y_true:y_true})
    assert (tf_hss_score_tau30_v - hss_score_tau30) < 1e-5

## Step Weight Function

$$ w(\text{y_true}) = \begin{cases}
    1,              & \text{if y_true < 2 or y_true =255 } \\
    2,              & \text{if 2} \leq \text{ y_true } < 5 \\
    5,              & \text{if 5} \leq \text{ y_true } < 10 \\
    10,              & \text{if 10} \leq \text{ y_true } < 30 \\
    30,              & \text{if 30} \leq \text{ y_true } \leq 80 
\end{cases} $$

In [7]:
MAX_VALID_PIXEL_VALE = 80
MISSING_PIXEL_VALUE = 255

## Loss calculation
def normal_y(y_val):
    """
    recover value from the (0,1) to [0, 80] with 255 as missing value, now only return integer value in y_val
    # MISSING_PIXEL_VALUE = 255
    # MAX_VALID_PIXEL_VALE = 80    
    """
    assert y_val.dtype == tf.float32
    y_val = tf.cast(y_val, tf.int32)
    y_val = tf.where(y_val > MAX_VALID_PIXEL_VALE, tf.fill(tf.shape(y_val), MISSING_PIXEL_VALUE), y_val)
    # the value > 80 should now > 255, then using tf.clip_by_value
    return y_val

def b_mse(y_true, y_pred):
    """
    B-MSE loss with step function
    """
    y_pred = tf.cast(normal_y(y_pred), dtype=tf.float32)
    N = tf.cast(tf.shape(y_true)[0], dtype=tf.float32)
    value = tf.square(y_true - y_pred)
    y_true_w1 = tf.cast(tf.logical_or(y_true < 2, y_true == 255), dtype=tf.float32)
    y_true_w2 = tf.cast(tf.logical_or(y_true >= 2, y_true < 5), dtype=tf.float32) * 2
    y_true_w3 = tf.cast(tf.logical_or(y_true >= 5, y_true < 10), dtype=tf.float32) * 5
    y_true_w4 = tf.cast(tf.logical_or(y_true >= 10, y_true < 30), dtype=tf.float32) * 10
    y_true_w5 = tf.cast(tf.logical_or(y_true >= 30, y_true <= 80), dtype=tf.float32) * 30
    weighted_value = value * y_true_w1 + value * y_true_w2 + value * y_true_w3 + value * y_true_w4 + value * y_true_w5
    return tf.reduce_sum(weighted_value) / N

def b_mae(y_true, y_pred):
    """
    B-MAE loss with step function
    """
    y_pred = tf.cast(normal_y(y_pred), dtype=tf.float32)
    N = tf.cast(tf.shape(y_true)[0], dtype=tf.float32)
    value = tf.abs(y_true - y_pred)
    y_true_w1 = tf.cast(tf.logical_or(y_true < 2, y_true == 255), dtype=tf.float32)
    y_true_w2 = tf.cast(tf.logical_or(y_true >= 2, y_true < 5), dtype=tf.float32) * 2
    y_true_w3 = tf.cast(tf.logical_or(y_true >= 5, y_true < 10), dtype=tf.float32) * 5
    y_true_w4 = tf.cast(tf.logical_or(y_true >= 10, y_true < 30), dtype=tf.float32) * 10
    y_true_w5 = tf.cast(tf.logical_or(y_true >= 30, y_true <= 80), dtype=tf.float32) * 30
    weighted_value = value * y_true_w1 + value * y_true_w2 + value * y_true_w3 + value * y_true_w4 + value * y_true_w5
    return tf.reduce_sum(weighted_value) / N

In [8]:
tf_b_mse = b_mse(tf_y_true, tf_y_pred)
tf_b_mae = b_mae(tf_y_true, tf_y_pred)

In [9]:
with tf.Session() as sess:
    tf_b_mse_v, tf_b_mae_v = sess.run([tf_b_mse, tf_b_mae], feed_dict={tf_y_pred:y_pred, tf_y_true:y_true})
    print("b_mse(y_true, y_pred) = %.2f" % (tf_b_mse_v))
    print("b_mae(y_true, y_pred) = %.2f" % (tf_b_mae_v))

b_mse(y_true, y_pred) = 17665694.00
b_mae(y_true, y_pred) = 11522720.00
